# Bonus : sélectionner un document dans la DataTable et l'afficher avec IIIF

Bonjour 👋 !

Bienvenue dans cette partie bonus dédiée à la création d'un second ***dashboard*** dédié à l'affichage des graphiques construits durant la partie 1, avec une fonctionnalité supplémentaire : **la possibilité d'ajouter ou retirer la courbe d'un journal du graphique !**.



# A/ Construction du *layout*

Pour vous aider, le fichier `illustrations.py` contient déjà un brouillon de *dashboard* à compléter.

Son *layout* est composé d'un composant titre `dash.html.H1`que l'on a déjà vu, mais aussi d'un nouveau composant `dash.dcc.Checklist`.  C'est une liste à cocher, très proche dans son fonctionnement du menu déroubale `dash.dcc.Dropdown` à la seule différence notable qu'il est possible de sélectionner **plusieurs options** en les cochant.

Il manque à ce *layout* un composant : le graphique ! 😮 Encore une fois, Dash met à disposition un composant "tout prêt" ; `dash.dcc.Graph`.

Ajoutez donc un composant `Graph` au *layout* et donnez-lui l'identifiant `id="graphe-illustrations"`.


## Chargement des données

Comme dans le premier *dashboard*, il nous faut charger les données sous forme de `DataFrame` depuis le fichier CSV `presse_xix-xxe.csv`.

Pour obtenir le même type de graphique que celui construit dans la partie 1, nous avons également besoin de rééchantillonner les dates à l'année.

Pour souci de concision, nous pouvons faire toute l'opération en une seule fois au chargement : 

```python
data = (
    # 1. On précise qu'il faut reconnaitre les élements de la colonne "date" comme...des dates
    pd.read_csv("presse_xix-xxe.csv", parse_dates=["date"])
    # 2. On chaîne en groupant par la colonne "titre" et "date"
    .groupby(["titre", pd.Grouper(key="date", freq="YE")])
    # 3. Puis on fait la somme des lignes groupées par année
    .sum()
    # 4. Et enfin on rétablit "titre" et "date" comme colonne, la méthode groupby() les ayant automatiquement transfromé en index.
    .reset_index()
)
```
<span style="color: #40d6d1"><strong>💡 Astuce</strong></span> Ce chaînage d'opérations vous semble cryptique et incompéhensible ? Voyons ça ensemble !  🆘 🙋


Une fois les données chargées, vous pouvez "remplir" les options du composant `Checklist` exactement comme pour `Dropdown`, c'est-à-dire en récupérant la liste des titres de presse à partir de `data` et en passant cette liste au composant : 
```python
dash.dcc.Checklist(
            id="selecteur-titre",
            options=data.titre.unique(),
            value=[],  # Aucun titre sélectionné par défaut
        ),
```

## Une fonction pour ajouter une courbe au graphique

Reste le coeur du *dashboard* : la mise à jour dynamique du graphique à partir des titres de presses cochés.

Comme pour lautre *dashboard*, créons une fonction *callback* qui sera déclenché par une sélection dans la liste à cocher et mettra à jour le graphique avec les courbes des titres sélectionnés.


Cette fonction - appelons la `tracer_courbes()` - a besoin de déclarer un unique paramètre qui sera la liste des titres de journaux  cochés : 

```python
def tracer_courbes(selection: list[str]):
    ...
```

Pour le crops de la fonction, l'idée générale est de créer un graphique "vide", puis de lui ajouter les courbes des journaux sélectionnés une par une. À la fin il suffira de renvoyer ce graphique enrichi.

Décomposons étape par étape :
1. D'abord, créer un graphique vide et le stocker dans une variable locale : `fig = px.line()` ;
2. Ensuite, parcourir la liste `selection`des titres de presse cochés et, pour chacun :
   1. Récupérer la partie de la table `data` correspondante  ; `data_titre = data[data["titre"] == titre]`
   2. Ajouter la courbe correspondante à `fig` ;
3. Retourner `fig`.

Pour ajouter une courbe à un graphique existant, on doit utiliser la méthode `fig.add_scatter()` :
```python
fig.add_scatter(x=data_titre.date, y=data_titre.nb_illustrations, mode="lines")
# Attention : contrairement à px.line() dont les paramètres x= et y= acceptaient des chaînes de caractère, ici ce sont directement les colonnes de données !
```
Oui, utiliser une méthode nommée [`add_scatter()`](https://plotly.com/python/creating-and-updating-figures/) pour ajouter une courbe porte vraiment à confusion...mais c'est comme ça pour l'instant dans Plotly 😕

Enfin, comme dans le pour le *dashboard* `dataset.py`, il faut encore déclarer l'entête *callback* pour que `tracer_courbe()` soit effectivement utilisée par Dash :

```python
# Déclaration du callback
# En sortie : on met à jour le graphique
 # En entrée : on écoute les changements de sélection dans la checklist
@dash.callback(
    dash.Output("graphe-illustrations", "figure"),  
    [dash.Input("selecteur-titre", "value")], 
)
def tracer_courbes(selection: list[str]):
    ...
```

<span style="color: #40d6d1"><strong>💡 Astuce</strong></span> Besoin d'une explication sur ce *callback* ? Voyons ça ensemble !  🆘 🙋